In [2]:
from lxml import etree
import requests
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.collections import PatchCollection
import matplotlib.font_manager as fm
from mpl_toolkits.basemap import Basemap
from shapely.geometry import Point, Polygon, MultiPoint, MultiPolygon, shape
from shapely.prepared import prep
from pysal.esda.mapclassify import Natural_Breaks as nb
from descartes import PolygonPatch
import mpl_toolkits.basemap.pyproj as pyproj
import fiona
from itertools import chain
%matplotlib inline

download indonesia gis shape files from [here](https://ordercitrasatelit.wordpress.com/2013/03/16/free-peta-batas-administrasi-desa-seluruh-indonesia/)

In [3]:
# select plot font
fm.findSystemFonts(fontpaths='/Library/Fonts/', fontext='ttf')
prop = fm.FontProperties(fname='/Library/Fonts/Times New Roman.ttf')